In [1]:
import numpy as np
import pandas as pd
from scipy.stats import norm
from scipy.optimize import brentq
import matplotlib.pylab as plt
import datetime as dt

In [2]:
rate_df = pd.read_csv('discount.csv')
call_df = pd.read_csv("goog_call.csv")
put_df = pd.read_csv("goog_put.csv")

In [3]:
today = dt.date(2013, 8, 30)
expiry = dt.date(2015, 1, 17)
T = (expiry-today).days/365.0
r = np.interp((expiry - today).days, rate_df['Day'], rate_df['Rate (%)'])/100

In [14]:
S0 = 846.9
F = S * np.exp(r*T)

In [5]:
call_df['mid'] = (call_df['best_bid'] + call_df['best_offer']) / 2
put_df['mid'] = (put_df['best_bid'] + put_df['best_offer']) / 2

In [6]:
def BlackScholesCall(S, K, r, sigma, T):
    d1 = (np.log(S/K)+(r+sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    return S*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)

def impliedCallVolatility(S, K, r, price, T):
    impliedVol = brentq(lambda x: price -
                        BlackScholesCall(S, K, r, x, T),
                        1e-6, 1)

    return impliedVol

In [7]:
def BlackScholesPut(S, K, r, sigma, T):
    d1 = (np.log(S/K)+(r+sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    return K*np.exp(-r*T)*norm.cdf(-d2) - S*norm.cdf(-d1)

def impliedPutVolatility(S, K, r, price, T):
    impliedVol = brentq(lambda x: price -
                        BlackScholesPut(S, K, r, x, T),
                        1e-6, 1)

    return impliedVol

In [8]:
# get the mid price of bid/offer as option price
put_df_option = put_df[put_df['strike'] <= F]
call_df_option = call_df[call_df['strike'] >= F]

In [9]:
call_df_option['impliedvol'] = call_df_option.apply(lambda x: impliedCallVolatility(S, x['strike'],r,  x['mid'], T), axis=1)
put_df_option['impliedvol'] = put_df_option.apply(lambda x: impliedPutVolatility(S, x['strike'],r,  x['mid'], T), axis=1)

C:\Users\mrrbi\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\mrrbi\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [10]:
df = pd.concat([put_df_option, call_df_option])

In [11]:
vol_at_strike = 0.5*(np.interp(F, put_df_option["strike"], put_df_option["impliedvol"])
                     +np.interp(F, call_df_option["strike"], call_df_option["impliedvol"]))
vol_at_strike

0.25812694845838613

#### Through Simulation

In [12]:
observations = 100
np.random.seed(0)
x = np.random.normal(0,1,observations)

#### Black Scholes Model

In [15]:
St = []
for i in x:
    price = S0*np.exp((r-0.5*(vol_at_strike**2))* T + vol_at_strike*np.sqrt(T)*i)
    St.append(price)
    
St = np.array(St)

In [16]:
payoff = []
for i in St:
    output = ((i ** 3)* 1e-8) + (0.5 * np.log(i)) + 10
    payoff.append(output) 
    
payoff = np.array(payoff)

In [17]:
Option_Value_BS = (payoff.mean())/np.exp(r*T)
Option_Value_BS

21.79437700234562

#### Bachelier Model

In [18]:
St_bach = []

for i in x:
    price = S0*(1+(vol_at_strike*np.sqrt(T)*i))
    St_bach.append(price)
    
St_bach = np.array(St_bach)

In [19]:
payoff_bach = ((St_bach ** 3)* 1e-8) + (0.5 * np.log(St_bach)) + 10
Option_Value_Bach = payoff_bach.mean()/np.exp(r*T)
Option_Value_Bach

21.38281200739997